# $facet 的基本操作

In [ ]:
# render several different facets using $facet stage
db.companies.aggregate([
    {
        $match: {
            $text: { $search: "Databases" }
        }
    },
    {
        $facet: {
            Categories: [{ $sortByCount: "$category_code" }],
            Employees: [
                { $match: { founded_year: { $gt: 1980 }}},
                {
                    $bucket: {
                        groupBy: "$number_of_employees",
                        boundaries: [0, 20, 50, 100, 500, 1000, Infinity],
                        default: "Other"
                    }
                }
            ],
            Founded: [
                { $match: { "offices.city": "New York" }},
                {
                    $bucketAuto: {
                        groupBy: "$founded_year",
                        buckets: 5
                    }
                }
            ]

        }
    }
]).pretty()

### 範例01

In [ ]:
# 參考解答
db.movies.aggregate([
    {
        $match: {
            metacritic: { $ne: null},
            "imdb.rating": { $gt: 0 }
        }
    },
    {
        $project:{
            title: 1,
            imdb_rating: "$imdb.rating",
            metacritic: 1
        }
    },
    {
        $facet: {
            imdbTop: [
                { $sort: { imdb_rating: -1 }},
                { $limit: 10 }
            ],
            metacriticTop: [
             { $sort: { metacritic: -1 }},
             { $limit: 10 }
            ]
        }
    },
    {
        $group: {
            _id: null,
            imdbTop: { $addToSet: "$imdbTop.title" },
            metacriticTop: { $addToSet: "$metacriticTop.title" }
        }
    },
    { $unwind: "$imdbTop" },
    { $unwind: "$metacriticTop" },
    {
        $project: {
            interSection: { $setIntersection: [ "$imdbTop", "$metacriticTop" ]}
        }
    }
]).pretty()

In [ ]:
# 參考解答
db.movies.aggregate([
  {
    $match: {
      metacritic: { $gte: 0 },
      "imdb.rating": { $gte: 0 }
    }
  },
  {
    $project: {
      _id: 0,
      metacritic: 1,
      imdb: 1,
      title: 1
    }
  },
  {
    $facet: {
      top_metacritic: [
        {
          $sort: {
            metacritic: -1,
            title: 1
          }
        },
        {
          $limit: 10
        },
        {
          $project: {
            title: 1
          }
        }
      ],
      top_imdb: [
        {
          $sort: {
            "imdb.rating": -1,
            title: 1
          }
        },
        {
          $limit: 10
        },
        {
          $project: {
            title: 1
          }
        }
      ]
    }
  },
  {
    $project: {
      movies_in_both: {
        $setIntersection: ["$top_metacritic", "$top_imdb"]
      }
    }
  }
]).pretty()